### Imports

In [2]:
from pyod.models.ecod import ECOD
from mpl_toolkits import mplot3d
from sklearn.metrics import silhouette_score
from sklearn.model_selection import RandomizedSearchCV
from tqdm import tqdm
from utils.utils import *

import hdbscan
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import os
import pandas as pd
import sys

%load_ext autotime

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 848 µs (started: 2023-06-12 21:39:35 -05:00)


### Parameters

In [3]:
import json
  
# Opening JSON file
file = open("parameters.json")
params = json.load(file)

exp = params["exp"]
print("Exp:\t\t", exp)

method = params["method"]
print("Method:\t\t", method)

dimension = params["dimension"]
print("Dimension:\t", dimension)

groups_id = params["groups_id"]
print("Groups id:\t", groups_id)

subgroups_id = params["subgroups_id"]
print("Subgroups id:\t", subgroups_id)

option = params["option"]
print("Option:\t\t", option)

if option:
    
    subgroups_id_op = {}
    for group in groups_id:
        subgroups_id_op[group] = [option]
else:
    subgroups_id_op = subgroups_id
print("Subgroups id op:", subgroups_id_op)



Exp:		 exp5
Method:		 dgi
Dimension:	 3
Groups id:	 ['s_WT', 's_zwf1^']
Subgroups id:	 {'s_WT': ['1', '2', '3', '4', '5'], 's_zwf1^': ['1', '2', '3']}
Option:		 
Subgroups id op: {'s_WT': ['1', '2', '3', '4', '5'], 's_zwf1^': ['1', '2', '3']}
time: 2.85 ms (started: 2023-06-12 21:39:35 -05:00)


### Edge embeddings

In [4]:
# get edges embeddings

edge_embeddings_global(exp, method, groups_id, subgroups_id_op)

 50%|█████     | 1/2 [01:01<01:01, 61.64s/it]

In [ ]:
df_edge_embeddings = pd.read_csv("output/{}/edge_embeddings/edge-embeddings_{}_{}_{}.csv".format(exp, method, groups_id[0], subgroups_id_op[groups_id[0]][0]), index_col=[0, 1])
df_edge_embeddings.head()

0             1             2
0 1  3.090705e-08  1.399957e-09  3.437019e-08
  2  3.211228e-08  1.454888e-09  3.570928e-08
  3  9.534077e-08  4.333721e-09  1.058793e-07
  4  2.844974e-07  1.295818e-08  3.156855e-07
  5  2.747919e-07  1.251333e-08  3.049414e-07

time: 536 ms (started: 2023-06-12 18:19:27 -05:00)


### Concat edge embeddings

In [ ]:
for group in tqdm(groups_id):
    df_edge_embeddings_concat = pd.DataFrame()
    k = 0
    for subgroup in tqdm(subgroups_id_op[group]):
        df_edge_embeddings = pd.read_csv("output/{}/edge_embeddings/edge-embeddings_{}_{}_{}.csv".format(exp, method, group, subgroup), index_col=[0, 1])
        df_edge_embeddings["subgroup"] = [k] * len(df_edge_embeddings)
        df_edge_embeddings_concat = pd.concat([df_edge_embeddings_concat, df_edge_embeddings])
        k += 1
    
    df_edge_embeddings_concat.to_csv("output/{}/edge_embeddings/edge-embeddings_concat_{}_{}_{}.csv".format(exp, method, group, option), index=True)

100%|██████████| 2/2 [00:11<00:00,  5.70s/it]

time: 11.4 s (started: 2023-06-12 18:19:28 -05:00)


In [ ]:
df_edge_embeddings_concat = pd.read_csv("output/{}/edge_embeddings/edge-embeddings_concat_{}_{}_{}.csv".format(exp, method, groups_id[0], option), index_col=[0, 1])
df_edge_embeddings_concat.head()

0             1             2  subgroup
0 1  3.090705e-08  1.399957e-09  3.437019e-08         0
  2  3.211228e-08  1.454888e-09  3.570928e-08         0
  3  9.534077e-08  4.333721e-09  1.058793e-07         0
  4  2.844974e-07  1.295818e-08  3.156855e-07         0
  5  2.747919e-07  1.251333e-08  3.049414e-07         0

time: 566 ms (started: 2023-06-12 18:19:39 -05:00)


In [ ]:
# plot edge embeddings concat

for group in tqdm(groups_id):
    df_edge_embeddings_concat = pd.read_csv("output/{}/edge_embeddings/edge-embeddings_concat_{}_{}_{}.csv".format(exp, method, group, option), index_col=[0, 1])

    x = df_edge_embeddings_concat.iloc[:, 0]
    y = df_edge_embeddings_concat.iloc[:, 1]
    z = df_edge_embeddings_concat.iloc[:, 2]

    # Creating figure
    fig = plt.figure(figsize=(10, 7))
    ax = plt.axes(projection="3d")

    # Creating plot
    ax.scatter3D(x, y, z, c=df_edge_embeddings_concat.iloc[:, -1], alpha=0.1)
    # plt.title("Dimension: {}".format(dimension))

    # show plot
    plt.savefig("output/{}/plots/edge-embeddings_concat_{}_{}_{}.png".format(exp, method, group, option))
    # plt.show()
    plt.clf()

100%|██████████| 2/2 [00:51<00:00, 25.61s/it]


<Figure size 1000x700 with 0 Axes>

<Figure size 1000x700 with 0 Axes>

time: 51.2 s (started: 2023-06-12 18:19:40 -05:00)


### Outliers detection

In [ ]:
# Outlier detection (HDBSCAN)

""" df_edge_embeddings_concat = pd.read_csv("output/edge_embeddings/edge-embeddings_concat_{}_{}_{}_{}.csv".format(group, method, dimension, "L2"), index_col=[0, 1])

X_train = df_edge_embeddings_concat.iloc[:, :-1]
clusterer = hdbscan.HDBSCAN(min_cluster_size=100, core_dist_n_jobs=-1).fit(X_train)

threshold = pd.Series(clusterer.outlier_scores_).quantile(0.9)
outliers = np.where(clusterer.outlier_scores_ > threshold)[0]
print(len(outliers))
outliers

inliers = np.setdiff1d(np.arange(len(df_edge_embeddings_concat)), outliers)
print(len(inliers))
inliers """

' df_edge_embeddings_concat = pd.read_csv("output/edge_embeddings/edge-embeddings_concat_{}_{}_{}_{}.csv".format(group, method, dimension, "L2"), index_col=[0, 1])\n\nX_train = df_edge_embeddings_concat.iloc[:, :-1]\nclusterer = hdbscan.HDBSCAN(min_cluster_size=100, core_dist_n_jobs=-1).fit(X_train)\n\nthreshold = pd.Series(clusterer.outlier_scores_).quantile(0.9)\noutliers = np.where(clusterer.outlier_scores_ > threshold)[0]\nprint(len(outliers))\noutliers\n\ninliers = np.setdiff1d(np.arange(len(df_edge_embeddings_concat)), outliers)\nprint(len(inliers))\ninliers '

time: 3.18 ms (started: 2023-06-12 18:20:31 -05:00)


In [ ]:
# outlier detection (ECOD)

dict_df_edge_embeddings_concat_outlier = {}
dict_df_edge_embeddings_concat_filter = {}

for group in tqdm(groups_id):
    df_edge_embeddings_concat = pd.read_csv("output/{}/edge_embeddings/edge-embeddings_concat_{}_{}_{}.csv".format(exp, method, group, option), index_col=[0, 1])

    X_train = df_edge_embeddings_concat.iloc[:, :-1]

    clf = ECOD()
    clf.fit(X_train)

    X_train["labels"] = clf.labels_ # binary labels (0: inliers, 1: outliers)

    df_edge_embeddings_concat_filter = df_edge_embeddings_concat.copy()
    df_edge_embeddings_concat_filter["labels"] = clf.labels_
    df_edge_embeddings_concat_filter = df_edge_embeddings_concat_filter[df_edge_embeddings_concat_filter["labels"] == 0]
    df_edge_embeddings_concat_filter = df_edge_embeddings_concat_filter.iloc[:, :-1]

    dict_df_edge_embeddings_concat_outlier[group] = X_train
    dict_df_edge_embeddings_concat_filter[group] = df_edge_embeddings_concat_filter

  0%|          | 0/2 [00:00<?, ?it/s]

100%|██████████| 2/2 [00:02<00:00,  1.50s/it]

time: 3.01 s (started: 2023-06-12 18:20:31 -05:00)


In [ ]:
df_edge_embeddings_concat_filter = dict_df_edge_embeddings_concat_filter[groups_id[0]]
df_edge_embeddings_concat_filter.head()

0             1         2  subgroup
0 27  0.000002  9.779630e-08  0.000002         0
  30  0.000002  8.578397e-08  0.000002         0
  32  0.000003  1.209815e-07  0.000003         0
  33  0.000002  9.987875e-08  0.000002         0
  34  0.000002  8.665900e-08  0.000002         0

time: 12.4 ms (started: 2023-06-12 18:20:34 -05:00)


In [ ]:
# plot outliers/inliers

for group in tqdm(groups_id):
    fig = plt.figure(figsize = (10, 7))
    ax = plt.axes(projection ="3d")

    df_aux = dict_df_edge_embeddings_concat_outlier[group]
    print("Total:", len(df_aux))
    
    temp = df_aux[df_aux["labels"] == 0]
    x = temp.iloc[:, 0]
    y = temp.iloc[:, 1]
    z = temp.iloc[:, 2]
    ax.scatter3D(x, y, z, c="red", alpha=0.005)
    print("Num. of inliers:", len(temp))

    temp = df_aux[df_aux["labels"] == 1]
    x = temp.iloc[:, 0]
    y = temp.iloc[:, 1]
    z = temp.iloc[:, 2]
    ax.scatter3D(x, y, z, c="gray", alpha=0.005)
    print("Num. of inliers:", len(temp))

    # show plot
    plt.savefig("output/{}/plots/edge-embeddings_outlier_{}_{}_{}.png".format(exp, method, group, option))
    # plt.show()
    plt.clf()

  0%|          | 0/2 [00:00<?, ?it/s]

Total: 1251401
Num. of inliers: 1126261
Num. of inliers: 125140


 50%|█████     | 1/2 [00:29<00:29, 29.74s/it]

Total: 750347
Num. of inliers: 675312
Num. of inliers: 75035


100%|██████████| 2/2 [00:47<00:00, 23.83s/it]


<Figure size 1000x700 with 0 Axes>

<Figure size 1000x700 with 0 Axes>

time: 47.7 s (started: 2023-06-12 18:20:34 -05:00)


###  Filter common edges

In [ ]:
print(subgroups_id)
print(subgroups_id_op)

{'s_WT': ['1', '2', '3', '4', '5'], 's_zwf1^': ['1', '2', '3']}
{'s_WT': ['dyn'], 's_zwf1^': ['dyn']}
time: 300 µs (started: 2023-06-12 18:21:22 -05:00)


In [ ]:
# mapping idx with id

for group in tqdm(groups_id):
    dict_df_nodes = {}
    for subgroup in subgroups_id_op[group]:
        df_nodes = pd.read_csv("output/{}/preprocessing/graphs_data/nodes_data_{}_{}.csv".format(exp, group, subgroup),
                               dtype={"id": "string"})
        dict_df_nodes[subgroup] = df_nodes
    
    # mapping
    df_edge_embeddings_concat_filter = dict_df_edge_embeddings_concat_filter[group]
    list_index = []
    
    for row in tqdm(df_edge_embeddings_concat_filter.itertuples()):
        df_nodes = dict_df_nodes[subgroups_id_op[group][row[-1]]]
        list_index.append((df_nodes.iloc[row[0][0], -1], df_nodes.iloc[row[0][1], -1]))
    
    # set new index
    df_edge_embeddings_concat_filter.set_index([pd.Index(list_index)], inplace=True)

1126261it [00:30, 36829.16it/s]?it/s]
675312it [00:18, 36352.98it/s]:31, 31.92s/it]
100%|██████████| 2/2 [00:51<00:00, 25.66s/it]

time: 51.3 s (started: 2023-06-12 18:21:22 -05:00)


In [ ]:
df_edge_embeddings_concat_filter = dict_df_edge_embeddings_concat_filter[groups_id[0]]
df_edge_embeddings_concat_filter.head()

0             1         2  subgroup
A0 A52  0.000002  9.779630e-08  0.000002         0
   A59  0.000002  8.578397e-08  0.000002         0
   A67  0.000003  1.209815e-07  0.000003         0
   A68  0.000002  9.987875e-08  0.000002         0
   A69  0.000002  8.665900e-08  0.000002         0

time: 9.05 ms (started: 2023-06-12 18:22:13 -05:00)


In [ ]:
# format id
if option:
    for group in tqdm(groups_id):
        # format
        df_edge_embeddings_concat_filter = dict_df_edge_embeddings_concat_filter[group]
        list_index = []

        for row in tqdm(df_edge_embeddings_concat_filter.itertuples()):
            list_index.append((row[0][0][1:], row[0][1][1:]))
        
        # set new index
        df_edge_embeddings_concat_filter.set_index([pd.Index(list_index)], inplace=True)
        df_edge_embeddings_concat_filter

  0%|          | 0/2 [00:00<?, ?it/s]

1126261it [00:01, 1096353.17it/s]
675312it [00:00, 1051003.86it/s]2,  2.74s/it]
100%|██████████| 2/2 [00:04<00:00,  2.17s/it]

time: 4.35 s (started: 2023-06-12 18:22:13 -05:00)


In [ ]:
df_edge_embeddings_concat_filter = dict_df_edge_embeddings_concat_filter[groups_id[0]]
df_edge_embeddings_concat_filter.head()

0             1         2  subgroup
0 52  0.000002  9.779630e-08  0.000002         0
  59  0.000002  8.578397e-08  0.000002         0
  67  0.000003  1.209815e-07  0.000003         0
  68  0.000002  9.987875e-08  0.000002         0
  69  0.000002  8.665900e-08  0.000002         0

time: 9.47 ms (started: 2023-06-12 18:22:18 -05:00)


In [ ]:
# filter diferente edges

if option:
    for group in tqdm(groups_id):
        df_edge_embeddings_concat_filter = dict_df_edge_embeddings_concat_filter[group]
        df_edge_embeddings_concat_filter = df_edge_embeddings_concat_filter[df_edge_embeddings_concat_filter.index.get_level_values(0) != df_edge_embeddings_concat_filter.index.get_level_values(1)]
        dict_df_edge_embeddings_concat_filter[group] = df_edge_embeddings_concat_filter

100%|██████████| 2/2 [00:00<00:00, 12.10it/s]

time: 176 ms (started: 2023-06-12 18:22:18 -05:00)


In [ ]:
df_edge_embeddings_concat_filter = dict_df_edge_embeddings_concat_filter[groups_id[0]]
print(df_edge_embeddings_concat_filter.shape)
df_edge_embeddings_concat_filter.head()

(1123201, 4)


0             1         2  subgroup
0 52  0.000002  9.779630e-08  0.000002         0
  59  0.000002  8.578397e-08  0.000002         0
  67  0.000003  1.209815e-07  0.000003         0
  68  0.000002  9.987875e-08  0.000002         0
  69  0.000002  8.665900e-08  0.000002         0

time: 6.47 ms (started: 2023-06-12 18:22:18 -05:00)


In [ ]:
# count edges and filter by count

dict_df_edges_filter = {}
for group in tqdm(groups_id):
    # read
    df_edge_embeddings_concat_filter = dict_df_edge_embeddings_concat_filter[group]

    # format
    df_edge_embeddings_concat_filter.reset_index(inplace=True)
    df_edge_embeddings_concat_filter.rename(columns={"level_0": "source", "level_1": "target"}, inplace=True)
    df_edge_embeddings_concat_filter[["source", "target"]] = df_edge_embeddings_concat_filter[["source", "target"]].astype("string")
    
    # sort edges
    sort_df_edges(df_edge_embeddings_concat_filter)

    df_edge_embeddings_concat_filter = df_edge_embeddings_concat_filter[["source", "target"]].value_counts().to_frame()
    df_edge_embeddings_concat_filter.reset_index(inplace=True)
    df_edge_embeddings_concat_filter.columns = ["source", "target", "count"]
    
    # filter
    df_edge_embeddings_concat_filter = df_edge_embeddings_concat_filter[df_edge_embeddings_concat_filter["count"] == len(subgroups_id[group])]
    df_edge_embeddings_concat_filter = df_edge_embeddings_concat_filter.iloc[:, [0, 1]]
    dict_df_edges_filter[group] = df_edge_embeddings_concat_filter

  0%|          | 0/2 [00:00<?, ?it/s]

100%|██████████| 2/2 [00:02<00:00,  1.35s/it]

time: 2.71 s (started: 2023-06-12 18:22:18 -05:00)


In [ ]:
df_edges_filter = dict_df_edges_filter[groups_id[0]]
df_edges_filter.head()

,source,target
0,535,701
1,371,552
2,371,545
3,283,464
4,744,924


time: 8.31 ms (started: 2023-06-12 18:22:21 -05:00)


In [ ]:
df_edges_filter.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9557 entries, 0 to 9556
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   source  9557 non-null   object
 1   target  9557 non-null   object
dtypes: object(2)
memory usage: 224.0+ KB
time: 25.3 ms (started: 2023-06-12 18:22:21 -05:00)


In [ ]:
# change data type
for group in tqdm(groups_id):
    df_edges_filter = dict_df_edges_filter[group]
    df_edges_filter[["source", "target"]] = df_edges_filter[["source", "target"]].astype("string")

100%|██████████| 2/2 [00:00<00:00, 130.84it/s]

time: 29.1 ms (started: 2023-06-12 18:22:21 -05:00)


In [ ]:
df_edges_filter = dict_df_edges_filter[groups_id[0]]
df_edges_filter.head()

,source,target
0,535,701
1,371,552
2,371,545
3,283,464
4,744,924


time: 4.08 ms (started: 2023-06-12 18:22:21 -05:00)


In [ ]:
df_edges_filter.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9557 entries, 0 to 9556
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   source  9557 non-null   string
 1   target  9557 non-null   string
dtypes: string(2)
memory usage: 224.0 KB
time: 11.1 ms (started: 2023-06-12 18:22:21 -05:00)


In [ ]:
# get weight by subgroups

dict_df_edges_filter_weight = get_weight_global(dict_df_edges_filter, exp, groups_id, subgroups_id)
df_edges_filter_weight = dict_df_edges_filter_weight[groups_id[0]]
df_edges_filter_weight.head()

  0%|          | 0/2 [00:00<?, ?it/s]

100%|██████████| 2/2 [00:03<00:00,  1.80s/it]


,source,target,subgroup1,subgroup2,subgroup3,subgroup4,subgroup5
8897,0,112,0.804566,0.732804,0.584968,0.601476,0.831138
8037,0,186,0.531764,0.998024,0.513954,0.871659,0.885208
7713,0,190,0.764557,0.815738,0.655281,0.632235,0.657801
7761,0,208,0.717383,0.547421,0.812064,0.722347,0.689631
8863,0,253,0.964549,0.846407,0.678148,0.694898,0.538027


time: 3.62 s (started: 2023-06-12 18:22:21 -05:00)


In [ ]:
df_edges_filter_weight = dict_df_edges_filter_weight[groups_id[0]]
df_edges_filter_weight.head()

,source,target,subgroup1,subgroup2,subgroup3,subgroup4,subgroup5
8897,0,112,0.804566,0.732804,0.584968,0.601476,0.831138
8037,0,186,0.531764,0.998024,0.513954,0.871659,0.885208
7713,0,190,0.764557,0.815738,0.655281,0.632235,0.657801
7761,0,208,0.717383,0.547421,0.812064,0.722347,0.689631
8863,0,253,0.964549,0.846407,0.678148,0.694898,0.538027


time: 5.93 ms (started: 2023-06-12 18:22:24 -05:00)


### Filter by STD and average weight

In [ ]:
dict_df_common_edges = std_global(dict_df_edges_filter_weight, exp, method, groups_id, option, th=0.3, plot=True, save=True)
dict_df_common_edges[groups_id[0]].head()

 50%|█████     | 1/2 [00:00<00:00,  3.56it/s]

100%|██████████| 2/2 [00:01<00:00,  1.93it/s]


,source,target,weight
0,0,112,0.710990
1,0,186,0.760122
2,0,190,0.705122
3,0,208,0.697769
4,0,253,0.744406


<Figure size 640x480 with 0 Axes>

time: 1.06 s (started: 2023-06-12 18:22:24 -05:00)


In [ ]:
df_common_edges = pd.read_csv("output/{}/common_edges/common_edges_{}_{}_{}.csv".format(exp, method, groups_id[0], option),
                              dtype={"source": "string", "target": "string"})
df_common_edges.head()

,source,target,weight
0,0,112,0.710990
1,0,186,0.760122
2,0,190,0.705122
3,0,208,0.697769
4,0,253,0.744406


time: 8.6 ms (started: 2023-06-12 18:22:25 -05:00)


In [ ]:
# show details

for group in tqdm(groups_id):
    df_common_edges = pd.read_csv("output/{}/common_edges/common_edges_{}_{}_{}.csv".format(exp, method, group, option))
    
    G = nx.from_pandas_edgelist(df_common_edges, "source", "target", edge_attr=["weight"])
    print("Group: {}".format(group))
    graph_detail(G)

100%|██████████| 2/2 [00:00<00:00, 20.30it/s]

Group: s_WT
Num. nodes: 990
Num. edges: 9557

Group: s_zwf1^
Num. nodes: 1000
Num. edges: 46428

time: 107 ms (started: 2023-06-12 18:22:25 -05:00)
